In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import librosa, librosa.display
import IPython.display as ipd
from scipy.io import wavfile 
from sklearn.svm import SVC

from google.colab import drive
import pickle
import io

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Загрузка данных
def loading_data():
  data = pd.read_csv('drive/My Drive/Audio analytics/train.csv', sep=',')
  data_wav = []
  for filename in data.fname:
    y, _ = librosa.load('drive/My Drive/Audio analytics/train/' + filename, sr=8000)
    data_wav.append(y)
  
  return data_wav, data.label

In [0]:
# Составление словаря с индексами классов
def class_indices(y):
  dictionary = dict()
  for i in set(y):
    dictionary[i] = []
  
  for i, cl in enumerate(y):
    dictionary[cl].append(i)
  
  return dictionary

In [0]:
# Получение фич
def getting_features(data_wav, sr=8000):
  data_chroma = []
  for i in data_wav:
    data_chroma.append(librosa.feature.chroma_stft(i, sr))

  return data_chroma

In [0]:
# Разделение данных на train, test
def train_test(data_chroma, dict_labels, test_size=0.2):
  X_train = []
  y_train = []
  X_test = []
  y_test = []
  
  for keys, values in dict_labels.items():
    n = len(values)
    train_size = int(n*(1-test_size))
    
    for i in range(train_size):
      X_train.append(data_chroma[values[i]])
      y_train.append(keys)
    
    for i in range(train_size, n):
      X_test.append(data_chroma[values[i]])
      y_test.append(keys)

  return X_train, y_train, X_test, y_test

In [0]:
# Сохранение в формате pkl
def save_data(X_train, y_train, X_test, y_test):
  pickle.dump(X_train, open('drive/My Drive/Audio analytics/pickle/X_train.pkl', 'wb'))
  pickle.dump(y_train, open('drive/My Drive/Audio analytics/pickle/y_train.pkl', 'wb'))
  pickle.dump(X_test, open('drive/My Drive/Audio analytics/pickle/X_test.pkl', 'wb'))
  pickle.dump(y_test, open('drive/My Drive/Audio analytics/pickle/y_test.pkl', 'wb'))

In [0]:
# Проигрывание звука
def play_sound(y, label, sr):
  #plt.figure(figsize=(15, 5))
  #librosa.display.waveplot(y, sr, alpha=0.8)
  print(label)
  ipd.Audio(y, rate=sr)

In [0]:
X, y = loading_data()

In [0]:
indices = class_indices(y)

In [0]:
X_chroma = getting_features(X)

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [0]:
X_train, y_train, X_test, y_test = train_test(X_chroma, indices)

In [0]:
save_data(X_train, y_train, X_test, y_test)